In [41]:
# Main import block
import numpy as np
import pandas as pd
import requests
import cfscrape 
from bs4 import BeautifulSoup
import numpy as np
import datetime
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re


import time
from tqdm import tqdm

# Upload to google sheets
import gspread
#import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [42]:
##### Functions to maintain side manipulations #####

# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw*1000
    return int(date_raw)


# Function for uploading dataframes into the google docs
def google_upload(df, sheet_name):
    # Params used to connect to google api
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'macro-parser-lme-c2f2972b48fc.json', scope)  # security token
    gc = gspread.authorize(credentials)

    # Key params for connection to particular document
    spreadsheet_key = '1WhLiXRcdlkG7NCvHac9unC8ROt4lcbY7GxrOEdezZ9s'  # document id
    wks_name = sheet_name  # sheet name that we use
    df_array = df.to_numpy()
    df_new = pd.DataFrame(df_array, columns=df.columns)
    # df_new.reset_index(drop=True, inplace=True)
    d2g.upload(df_new, spreadsheet_key, wks_name, credentials=credentials)
    # print(f'Uploading to {sheet_name} completed')
    time.sleep(3)


# Session creation via proxy
def get_session(url):

    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find('table').find_all('tr')  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all('td'))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == 'yes':  # taking only https
                    proxies[raw_list[i]
                            [3].text] = f'{raw_list[i][0].text}:{raw_list[i][1].text}'
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f'Attempt {counter+1}')

    return response

### This is the main function block ###

In [43]:
################################################################
##########################   LME ###############################
################################################################

def lme_selenium():
    def get_data(metall=''):
        url = f'https://www.lme.com/Metals/Non-ferrous/LME-{metall}#Trading+day+summary'

        service = Service()
        driver = webdriver.Chrome(service=service)

        try:
            driver.maximize_window()
            driver.get(url=url)

            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);")
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);")

            time.sleep(2)

            html_code = driver.page_source

            soup = BeautifulSoup(html_code, 'html.parser')
            data_raw = soup.find_all('td', class_='data-set-table__cell')
            price = pd.to_numeric(data_raw[1].text)

            # Date extraction
            date_raw = (soup.find_all(
                'span', class_='data-set-tabs__content-detail')[2].text)
            date_raw = date_raw.strip()
            date_list = re.sub(r'\W+', ' ', date_raw)
            date_list = date_list.split(' ')
            date = date_list[3:]
            date = ' '.join(date)
            date = pd.to_datetime(date)

            return (date, price)

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    date, _ = get_data('Aluminium')
    _, al_price = get_data('Aluminium')
    _, cu_price = get_data('Copper')
    _, zn_price = get_data('Zinc')
    _, nk_price = get_data('Nickel')
    _, ld_pric = get_data('Lead')

    data_row = pd.DataFrame({
        'date': date,
        'aluminium': al_price,
        'copper': cu_price,
        'lead': ld_pric,
        'nickel': nk_price,
        'zink': zn_price
    }, index=[0])

    historical_data = pd.read_excel(
        './data/LME_db.xlsx', index_col=0)

    historical_data = pd.concat(
        [historical_data, data_row]).reset_index(drop=True)

    historical_data.drop_duplicates(inplace=True)

    with pd.ExcelWriter('./data/LME_db.xlsx', date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        historical_data.to_excel(writer, sheet_name='LME_non_ferrous')
    
    google_upload(historical_data, 'LME_non_ferrous')
    
    return historical_data

In [44]:
################################################################
#########################   KITCO ##############################
################################################################

def kitco_db():
    # In KITCO parsing we're taking slightly different aproach, we need to reupload
    # the table into the file because sometimes KITCO changing data backdating

    #!!!Need to work out the implementation of previous year data!!!!
    # year = int(date.today().year) - 2001

    url = 'https://www.kitco.com/gold.londonfix.html'
    # url_previous_year = f'https://www.kitco.com/londonfix/gold.londonfix{year}.html'

    # response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) # ordinal responce
    response = get_session(url=url)  # response via proxy
    # responce_prev = requests.get(url_previous_year, headers={'User-Agent': 'Mozilla/5.0'})

    kitco_response = pd.read_html(response.text)
    kitco_df = pd.read_excel(
        '../parser_beta/data/kitko_db.xlsx', index_col=0)

    # Get the raw table and drop unnesesary rows
    kitco_day = kitco_response[1]
    kitco_day.drop([1, 4, 6], axis=1, inplace=True)
    kitco_day.columns = kitco_day.iloc[0]
    kitco_day.drop([0, 1, 2], axis=0, inplace=True)

    # Change tyoe of data within table
    kitco_day['Date'] = pd.to_datetime(kitco_day['Date'])
    kitco_day = kitco_day.replace({'-': np.nan})
    kitco_day = kitco_day.sort_values(by=['Date'])
    kitco_day = kitco_day.reset_index(drop=True)
    kitco_day[['Gold', 'Silver', 'Platinum', 'Palladium']] = kitco_day[[
        'Gold', 'Silver', 'Platinum', 'Palladium']].apply(pd.to_numeric)

    kitco_day.drop_duplicates(inplace=True)

    # And rewrite old table
    kitco_day.to_excel(
        '../parser_beta/data/kitko_db.xlsx', sheet_name='kitco_metall')
    # print('KITCO parsing is DONE!')

    google_upload(kitco_day, 'KITCO')

    return kitco_day

In [45]:
################################################################
#########################   CB #################################
################################################################

def cb_curr():
    day = date.today()
    today = day.strftime('%d/%m/%Y')

    dict_of_currencies = {
        'R01235': 'USD',
        'R01239': 'EUR',
        'R01010': 'Australian_Dollar',
        'R01375': 'China_Yuan',
        'R01035': 'British_Pound',
        'R01335': 'Kazakhstan_Tenge',
        'R01820': 'Japanese_Yen',
        'R01775': 'Swiss_Franc'
    }

    list_of_currencies = [x for x in dict_of_currencies.keys()]

    URL_list = []
    for currency in list_of_currencies:
        URL = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={today}&date_req2={today}&VAL_NM_RQ={currency}'
        URL_list.append(URL)

    currency_df = pd.read_excel(
        '../parser_beta/data/cb_curr.xlsx', index_col=0)

    # This problem occurs in the beginning of the year so I was forced to catch a ValueError

    try:
        for url_element in URL_list:
            response_df = pd.read_xml(url_element)
            response_df['Date'] = pd.to_datetime(
                response_df['Date'], dayfirst=True)
            response_df['Value'] = response_df['Value'].apply(
                lambda x: x.replace(',', '.'))
            response_df['Value'] = response_df['Value'].apply(pd.to_numeric)
            response_df.drop(columns=['VunitRate'], axis=1, inplace=True)
            response_df = response_df.replace(dict_of_currencies)
            currency_df = pd.concat(
                [currency_df, response_df], axis=0, ignore_index=True)
            currency_df.drop_duplicates(inplace=True)
        # print('CentroBank_currency parsing is DONE!')

        with pd.ExcelWriter(
                "../parser_beta/data/cb_curr.xlsx") as writer:
            currency_df.to_excel(writer, sheet_name='curr')

        google_upload(currency_df, 'cb_curr')

    except ValueError:
        return 'Empty data set in CentroBank_currency. Should check the source.'

    return currency_df


def cb_metall():
    day = date.today()
    today = day.strftime('%d/%m/%Y')

    metall_dict = {
        1: 'gold',
        2: 'silver',
        3: 'platinum',
        4: 'palladium'
    }

    URL = f'http://www.cbr.ru/scripts/xml_metall.asp?date_req1={today}&date_req2={today}'

    metall_df = pd.read_excel(
        '../parser_beta/data/cb_metall.xlsx', index_col=0)

    # This problem occurs in the beginning of the year so I was forced to catch a ValueError
    try:
        response_df = pd.read_xml(URL)
        response_df.drop(columns='Buy', axis=1, inplace=True)
        response_df['Date'] = pd.to_datetime(
            response_df['Date'], dayfirst=True)
        response_df['Sell'] = response_df['Sell'].apply(
            lambda x: x.replace(',', '.'))  # changing for future retyping to numeric
        response_df['Sell'] = response_df['Sell'].apply(pd.to_numeric)
        response_df = response_df.replace(metall_dict)

        metall_df = pd.concat(
            [metall_df, response_df],
            axis=0,
            ignore_index=True)

        metall_df.drop_duplicates(inplace=True)
        # print('CentroBank_metalls parsing is DONE!')

        with pd.ExcelWriter(
            '../parser_beta/data/cb_metall.xlsx',
            date_format='YYYY-MM-DD',
                datetime_format='YYYY-MM-DD') as writer:
            metall_df.to_excel(writer, sheet_name='cb_metall')

        google_upload(metall_df, 'cb_metall')

    except ValueError:
        return 'Empty data set in CentroBank_metalls . Should check the source.'

    return metall_df

In [46]:
################################################################
#########################   NBK ################################
################################################################

def nbk_tenge():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    year = date.today().year

    upper_bound = f'01.01.{year}'
    lower_bound = f'31.12.{year}'

    url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
        /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

    counter = 0

    while counter <= 6:
        try:
            with time_limit(15):
                page = requests.get(url=url)
                break

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1

    temp_df = pd.read_html(page.text)
    df = temp_df[0]
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])  # , dayfirst=True)
    df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

    with pd.ExcelWriter(
            '../parser_beta/data/nbk_tenge.xlsx') as writer:
        df.to_excel(writer, sheet_name='tenge')

    # print('NBK_tenge parsing is DONE!')

    google_upload(df, 'nbk_tenge')

    return df

In [47]:
################################################################
##########################  SHMET ##############################
################################################################

def shmet_optimized():
    url = 'https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0'
    responce = requests.get(url)

    day_df = pd.DataFrame(responce.json()['data'])
    day_df['date'] = date.today()
    cooper_row = day_df[day_df['name'].str.contains("cu", case=False)]

    result = cooper_row[['date', 'middle', 'unit']]
    result['date'] = pd.to_datetime(result['date'])
    result = result.rename(columns={'middle': 'price'})

    hist_data = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)

    new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
    new_df.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        new_df.to_excel(writer, sheet_name='SHMET')

    final = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    final.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final.to_excel(writer, sheet_name='SHMET')

    google_upload(final, 'SHMET')

    return final

In [48]:
################################################################
################## NEW_WESTMETALL###############################
################################################################

def new_westmetall():

    def get_data(metall, col_name):
        url = f'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash'
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data['date'] = pd.to_datetime(data['date'])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    al = get_data(metall='Al', col_name='aluminium')
    cu = get_data(metall='Cu', col_name='copper')
    pb = get_data(metall='Pb', col_name='lead')
    ni = get_data(metall='Ni', col_name='nickel')
    zn = get_data(metall='Zn', col_name='zink')

    result = pd.merge(al, cu, on='date', how='left').merge(pb, on='date', how='left').merge(
        ni, on='date', how='left').merge(zn, on='date', how='left')

    old_data = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)

    final_data = pd.concat([old_data, result], axis=0)

    final_data.drop_duplicates(subset='date', inplace=True)

    final_data.sort_values(by='date', inplace=True)

    final_data.reset_index(inplace=True, drop=True)

    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final_data.to_excel(writer, sheet_name='LME_westmetall')

    google_upload(final_data, 'LME_westmetall')

    return final_data

In [49]:
################################################################
####################   INVESTING.COM ###########################
################################################################

def investing_com():

    def get_data(url, table_idx=1, currency=False):
        today = pd.to_datetime(datetime.now().date())

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        data_frame = pd.read_html(scraped_data.text)[table_idx]

        data_frame.rename(
            columns={'Дата': 'date', 'Цена': 'price'}, inplace=True)

        result = data_frame[['date', 'price']]
        result = pd.DataFrame(result)

        if currency:
            result['price'] = result['price']/10000

        else:

            try:
                result['price'] = result['price'].str.replace('.', '')
                result['price'] = result['price'].str.replace(',', '.')
                result['price'] = pd.to_numeric(result['price'])
            except:
                pass

        result['date'] = pd.to_datetime(result['date'], dayfirst=True)

        result = result[result['date'] != today]

        result.drop_duplicates(inplace=True)
        result.reset_index(inplace=True, drop=True)

        return result

    ################################################################
    ########################### GDAXI ##############################
    ################################################################

    gdaxi = get_data(
        'https://ru.investing.com/indices/germany-30-historical-data')

    historical_data_gdaxi = pd.read_excel(
        './data/investing_com/GDAXI.xlsx', index_col=0)

    gdaxi_final = pd.concat([gdaxi, historical_data_gdaxi], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/GDAXI.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        gdaxi_final.to_excel(writer, sheet_name='GDAXI')

    google_upload(gdaxi_final, 'GDAXI')

    ################################################################
    ########################### IMOEX ##############################
    ################################################################

    imoex = get_data(
        'https://ru.investing.com/indices/mcx-historical-data', table_idx=0)

    historical_data_imoex = pd.read_excel(
        './data/investing_com/IMOEX.xlsx', index_col=0)

    imoex_final = pd.concat([imoex, historical_data_imoex], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/IMOEX.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        imoex_final.to_excel(writer, sheet_name='IMOEX')

    google_upload(imoex_final, 'IMOEX')

    ################################################################
    ########################### SPX ################################
    ################################################################

    spx = get_data(
        'https://ru.investing.com/indices/us-spx-500-historical-data')

    historical_data_spx = pd.read_excel(
        './data/investing_com/SPX.xlsx', index_col=0)

    spx_final = pd.concat([spx, historical_data_spx], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/SPX.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        spx_final.to_excel(writer, sheet_name='SPX')

    google_upload(spx_final, 'SPX')

    ################################################################
    ########################### SSEK ###############################
    ################################################################

    ssek = get_data(
        'https://ru.investing.com/indices/shanghai-composite-historical-data', table_idx=0)

    historical_data_ssek = pd.read_excel(
        './data/investing_com/SSEC.xlsx', index_col=0)

    ssek_final = pd.concat([ssek, historical_data_ssek], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/SSEK.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        ssek_final.to_excel(writer, sheet_name='SSEC')

    google_upload(ssek_final, 'SSEC')

    ################################################################
    ########################### URALS ##############################
    ################################################################

    urals = get_data(
        'https://ru.investing.com/commodities/crude-oil-urals-spot-futures-historical-data', table_idx=0)

    historical_data_urals = pd.read_excel(
        './data/investing_com/urals.xlsx', index_col=0)

    urals_final = pd.concat([urals, historical_data_urals], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/urals.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        urals_final.to_excel(writer, sheet_name='urals')

    google_upload(urals_final, 'urals')

    ################################################################
    ########################### EUR/USD ############################
    ################################################################

    currency = get_data(
        'https://ru.investing.com/currencies/eur-usd-historical-data', currency=True)

    historical_data_currency = pd.read_excel(
        './data/investing_com/curr.xlsx', index_col=0)

    currency_final = pd.concat([currency, historical_data_currency], axis=0).reset_index(
        drop=True).drop_duplicates('date')

    with pd.ExcelWriter("./data/investing_com/curr.xlsx",
                        date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
        currency_final.to_excel(writer, sheet_name='curr_investing_com')

    google_upload(currency_final, 'curr_investing_com')

    return None

In [50]:
new_westmetall()

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
463,2023-11-03,2219.5,8070.5,2164.0,17810,2494.0
464,2023-11-06,2254.0,8136.5,2166.5,17865,2559.5
465,2023-11-07,2241.0,8068.0,2166.0,17585,2538.0
466,2023-11-08,2258.5,8088.0,2201.0,17865,2604.5


In [51]:
cb_curr()

,Date,Id,Nominal,Value
0,2022-12-28,USD,1,69.9346
1,2022-12-28,EUR,1,74.1829
2,2022-12-28,Australian_Dollar,1,46.7163
3,2022-12-28,China_Yuan,10,99.4575
4,2022-12-28,British_Pound,1,84.1313
...,...,...,...,...
1715,2023-11-10,China_Yuan,1,12.5998
1716,2023-11-10,British_Pound,1,113.0329
1717,2023-11-10,Kazakhstan_Tenge,100,19.6934
1718,2023-11-10,Japanese_Yen,100,60.8947


In [52]:
cb_metall()

,Date,Code,Sell
0,2022-12-28,gold,4048.78
1,2022-12-28,silver,53.37
2,2022-12-28,platinum,2237.21
3,2022-12-28,palladium,3727.93
4,2022-12-29,gold,4129.34
...,...,...,...
855,2023-11-09,palladium,3171.70
856,2023-11-10,gold,5790.87
857,2023-11-10,silver,66.32
858,2023-11-10,platinum,2600.85


In [53]:
nbk_tenge()

,date,Числовое значение,ДОЛЛАР США
0,2023-01-01,1,462.65
1,2023-01-02,1,462.65
2,2023-01-03,1,462.65
3,2023-01-04,1,462.65
4,2023-01-05,1,465.39
...,...,...,...
309,2023-11-06,1,465.14
310,2023-11-07,1,462.46
311,2023-11-08,1,464.49
312,2023-11-09,1,466.79


In [54]:
kitco_db()


Attempt 2
Attempt 3
Attempt 4
Attempt 5
Attempt 6


,Date,Gold,Silver,Platinum,Palladium
0,2023-01-03,1843.25,24.295,1082.0,1795.0
1,2023-01-04,1857.30,24.290,1080.0,1736.0
2,2023-01-05,1834.00,23.410,1062.0,1783.0
3,2023-01-06,1852.20,23.455,1073.0,1784.0
4,2023-01-09,1878.85,23.850,1092.0,1793.0
...,...,...,...,...,...
213,2023-11-06,1984.60,23.210,927.0,1123.0
214,2023-11-07,1960.70,22.535,899.0,1070.0
215,2023-11-08,1959.35,22.440,880.0,1016.0
216,2023-11-09,1957.45,22.550,NaN,NaN


In [55]:
shmet_optimized()


,date,price,unit
0,2023-11-10,67750,Yuan/MT
1,2023-11-09,67930,Yuan/MT
2,2023-11-08,67930,Yuan/MT
3,2023-11-08,68025,Yuan/MT
4,2023-11-07,67810,Yuan/MT
...,...,...,...
734,2020-01-17,48930,Yuan/MT
735,2020-01-16,48950,Yuan/MT
736,2020-01-15,49060,Yuan/MT
737,2020-01-14,48990,Yuan/MT


In [56]:
lme_selenium()

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730.0,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900.0,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480.0,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725.0,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045.0,3576.5
...,...,...,...,...,...,...
463,2023-11-03,2219.5,8070.5,2164.0,17810.0,2494.0
464,2023-11-06,2254.0,8136.5,2166.5,17865.0,2559.5
465,2023-11-07,2241.0,8068.0,2166.0,17585.0,2538.0
466,2023-11-08,2258.5,8088.0,2201.0,17865.0,2604.5


In [57]:
investing_com()